<a href="https://colab.research.google.com/github/SAQUIBZEYA/Deep-Learning-Project/blob/main/Stackoverflow_Tag_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
!pip install -U -q PyDrive

In [3]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [8]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:

file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: stacksample.zip, id: 1gpQxAENOzeRLAMqqx7I71qSpQnjjWxJl
title: Saquib_CV (1), id: 1ZoKQEDXQjfFV8_yhxJl3f6WNclQ-cAbt4VYG3wOxM-4
title: TCOE%20First%20Week%20Plan.xlsx, id: 101dZvgHuZX_FcXmTnwir-OeU-3KgJgFX
title: Aadhaar card.pdf, id: 1EPKaH3GMbeO6-7LYokSsbXt8FaB_KUiC
title: Aadhaar card.pdf, id: 1fR7ANBBt4NniPaeapgxFWMUafHQ-fyDw
title: New Doc 2020-12-12.pdf, id: 15ku7WzbxILUH3mmpsVww9E7xN2ZvHgH-
title: Saquib_CV.pdf, id: 1Tk1Mzcu3gNEonWMJjr4Ca-OXcCKsT6R3
title: PIVOT TABLE.xlsx, id: 1w_SLi3woHboaVG2qxrB39kmGHdPhMVIA
title: Combine Data of All Sheets (1).xlsx, id: 1gyEjsEJYOj32G3NAWh9RAwd1UpXlFi5l
title: Untitled spreadsheet, id: 1IN2eqxeY9WePKNTRMArEXeTIaOZseHdGanqJ26cP6ms
title: Untitled form (Responses), id: 1-rdX2T49w_oOruPl3akE0UwTWWRd4fvLpeNtNkamLUw
title: Import HTML.xlsx, id: 175GdF363LftGFKFqL6u1Ri--vlVvrdnw
title: combine_workbook, id: 1bh8diCxwdk7crqGjGAZWTwk0Z9g7dAEeIjye4MxwR4s
title: Combine _workbook, id: 19bHH9zYFIRvtti8-Ld2HkmOrJx6zdnMrWBtrAGRS2O0
title: MAR_WORKB

In [6]:
!unzip /content/drive/MyDrive/stacksample.zip

Archive:  /content/drive/MyDrive/stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [12]:
import os

In [13]:

os.listdir()

['.config',
 'Tags.csv',
 'adc.json',
 'Questions.csv',
 'Answers.csv',
 'drive',
 'sample_data']

In [14]:

import warnings
warnings.filterwarnings('ignore')

In [15]:
import numpy as np
import pandas as pd

In [16]:
file=r'/content/Tags.csv'

tags=pd.read_csv(file)

In [17]:

tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [18]:
file1=r'/content/Questions.csv'

Questions=pd.read_csv(file1, encoding="ISO-8859-1")

In [19]:
Questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [20]:
tags['Tag'] = tags['Tag'].astype(str)

In [21]:

tags_grouped=tags.groupby('Id')['Tag'].apply(lambda tags: ' '.join(tags))

In [22]:

tags_grouped.reset_index()

,Id,Tag
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction
...,...,...
1264211,40143210,php .htaccess
1264212,40143300,google-bigquery
1264213,40143340,android android-studio
1264214,40143360,javascript vue.js


In [23]:
tags_grouped_final=pd.DataFrame({'Id':tags_grouped.index, 'Tags':tags_grouped.values})

In [24]:

tags_grouped_final.head()

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction


In [25]:
Questions.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'],inplace=True)

In [26]:
Questions=Questions.merge(tags_grouped_final,on='Id')

In [27]:
Questions.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction


In [28]:
Question=Questions[Questions['Score']>5]

In [29]:

Question.isnull().sum()

Id       0
Score    0
Title    0
Body     0
Tags     0
dtype: int64

In [30]:
Question.drop(columns=['Id','Score'],inplace=True)

In [32]:
Question.shape

(72950, 3)

In [33]:
Question['Tags']=Question['Tags'].apply(lambda x: x.split())

In [34]:
Question.head()

,Title,Body,Tags
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [35]:
all_tags = [item for sublist in Question['Tags'].values for item in sublist]

In [36]:
len(all_tags)

224129

In [37]:
my_set = set(all_tags)
unique_tags = list(my_set)
len(unique_tags)

14883

In [38]:
import nltk

In [39]:
flat_list = [item for sublist in Question['Tags'].values for item in sublist]

keywords = nltk.FreqDist(flat_list)

keywords = nltk.FreqDist(keywords)

frequencies_words = keywords.most_common(100)
tags_features = [word[0] for word in frequencies_words]

In [40]:
def most_common(tags):
    tags_filtered = []
    for i in range(0, len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [41]:
Question['Tags'] = Question['Tags'].apply(lambda x: most_common(x))
Question['Tags'] = Question['Tags'].apply(lambda x: x if len(x)>0 else None)

In [42]:
Question.shape

(72950, 3)

In [43]:
Question.dropna(subset=['Tags'], inplace=True)

In [44]:
Question.shape

(63167, 3)

In [45]:
from bs4 import BeautifulSoup

In [46]:
Question['Body']=Question['Body'].apply(lambda x : BeautifulSoup(x).get_text())

In [47]:
Question.head()

,Title,Body,Tags
1,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,[svn]
2,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]"
3,Function for creating color wheels,This is something I've pseudo-solved many time...,[algorithm]
4,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net]"
5,Should I use nested classes in this case?,I am working on a collection of classes used f...,"[c++, oop, class]"


In [48]:

Question['Body'].iloc[3]

"I have a little game written in C#. It uses a database as back-end. It's \na trading card game, and I wanted to implement the function of the cards as a script.\nWhat I mean is that I essentially have an interface, ICard, which a card class implements (public class Card056 : ICard) and which contains function that are called by the game.\nNow, to make the thing maintainable/moddable, I would like to have the class for each card as source code in the database and essentially compile it on first use. So when I have to add/change a card, I'll just add it to the database and tell my application to refresh, without needing any assembly deployment (especially since we would be talking about 1 assembly per card which means hundreds of assemblies).\nIs that possible? Register a class from a source file and then instantiate it, etc.\nICard Cards[current] = new MyGame.CardLibrary.Card056();\nCards[current].OnEnterPlay(ref currentGameState);\n\nThe language is C#, but extra bonus if it's possibl

In [49]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import ToktokTokenizer

In [50]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [51]:

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [52]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
token=ToktokTokenizer()

In [56]:

import re

In [57]:
punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [58]:
punct = '!#$%&\'()*+,./:;<=>?@[\\]^_{|}~'

In [59]:

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

In [60]:
def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in tags_features:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

In [61]:

Question['Body'] = Question['Body'].apply(lambda x: clean_punct(x))

In [62]:
lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [63]:
def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    
    stop_words = set(stopwords.words("english"))
    
    words=token.tokenize(text)
    
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))

In [64]:
Question['Body'] = Question['Body'].apply(lambda x: lemitizeWords(x)) 
Question['Body'] = Question['Body'].apply(lambda x: stopWordsRemove(x))

In [65]:

Question['Title'] = Question['Title'].apply(lambda x: clean_punct(x)) 
Question['Title'] = Question['Title'].apply(lambda x: lemitizeWords(x)) 
Question['Title'] = Question['Title'].apply(lambda x: stopWordsRemove(x))

In [66]:

Question['Body'].iloc[2]

'This something I pseudo-solved many time never quite find solution That stick The problem come way generate N color distinguishable possible N parameter'

In [67]:
Question['Title_Body']=Question['Title']+' '+ Question['Body']

In [68]:
Question.iloc[3,3]

'Adding script functionality NET applications I little game write C It use database back-end It trade card game I want implement function card script What I mean I essentially interface ICard card class implement public class Card056 ICard contain function call game Now make thing maintainablemoddable I would like class card source code database essentially compile first use So I addchange card I add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies Is possible Register class source file instantiate etc ICard Cards current new MyGameCardLibraryCard056 Cards current OnEnterPlay ref currentGameState The language C extra bonus possible write script NET language'

In [69]:
from sklearn.preprocessing import MultiLabelBinarizer
tag_encoder = MultiLabelBinarizer()
tags_encoded = tag_encoder.fit_transform(Question['Tags'])

tags_encoded.shape

(63167, 100)

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Question['Title_Body'], tags_encoded, test_size=0.2, random_state=42)

In [71]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from nltk import word_tokenize

In [72]:
sent_lens=[]
for sent in X_train:
    sent_lens.append(len(word_tokenize(sent)))

In [73]:
max(sent_lens)

3370

In [74]:
np.quantile(sent_lens,0.95)

306.0

In [75]:
# Word2Vec
import gensim

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# Collect corpus for training word embeddings
documents = [_text.split() for _text in np.array(Question['Title_Body'])]  # I used the entire corpus for training, because it contains the entire conversation of related to a user's posts


# Train Word Embeddings and save
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)
w2v_model.build_vocab(documents)
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)
# Train Word Embeddings
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)
w2v_model.save('SO_word2vec_embeddings.bin')

Vocab size 29278


In [76]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [77]:
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(Question['Title_Body'])
word_index = tokenizer.word_index
vocab_size = len(word_index)
print('Found %s unique tokens.' % len(word_index))


# Convert the data to padded sequences
X_train_padded = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_padded, maxlen=MAX_SEQUENCE_LENGTH)


# Embedding matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size+1, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

Found 375146 unique tokens.
(375147, 300)


In [78]:
def RNN():
    inputs = Input(name='inputs',shape=[MAX_SEQUENCE_LENGTH])
    layer = Embedding(vocab_size+1, W2V_SIZE, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)(inputs)
    layer = LSTM(100)(layer)
    layer = Dense(1000,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.35)(layer)
    layer = Dense(700,name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.25)(layer)
    layer = Dense(400,name='FC3')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(100,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
  
  
model = RNN()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 300)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 300)          112544100 
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
FC1 (Dense)                  (None, 1000)              101000    
_________________________________________________________________
activation (Activation)      (None, 1000)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
FC2 (Dense)                  (None, 700)               700700

In [79]:
X_test_padded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_padded, maxlen=MAX_SEQUENCE_LENGTH)

In [80]:
model.fit(X_train_padded,y_train,batch_size=1024,epochs=15,validation_data=(X_test_padded,y_test))

Epoch 1/15
50/50 [==============================] - 21s 248ms/step - loss: 0.2918 - accuracy: 0.0236 - val_loss: 0.0740 - val_accuracy: 0.0739
Epoch 2/15
50/50 [==============================] - 12s 234ms/step - loss: 0.0800 - accuracy: 0.0612 - val_loss: 0.0728 - val_accuracy: 0.0860
Epoch 3/15
50/50 [==============================] - 11s 230ms/step - loss: 0.0767 - accuracy: 0.0743 - val_loss: 0.0702 - val_accuracy: 0.1141
Epoch 4/15
50/50 [==============================] - 11s 229ms/step - loss: 0.0729 - accuracy: 0.1019 - val_loss: 0.0662 - val_accuracy: 0.1808
Epoch 5/15
50/50 [==============================] - 11s 228ms/step - loss: 0.0684 - accuracy: 0.1600 - val_loss: 0.0610 - val_accuracy: 0.2530
Epoch 6/15
50/50 [==============================] - 11s 230ms/step - loss: 0.0631 - accuracy: 0.2369 - val_loss: 0.0530 - val_accuracy: 0.3520
Epoch 7/15
50/50 [==============================] - 11s 229ms/step - loss: 0.0547 - accuracy: 0.3529 - val_loss: 0.0467 - val_accuracy: 0.4121

In [81]:

predictions=model.predict(X_test_padded)

In [82]:

from sklearn.metrics import roc_auc_score

In [83]:

roc_auc_score(y_test,predictions)

0.9307982978889919

In [85]:

def predict_tags(text):
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)
    # Predict
    prediction = model.predict([x_test])[0]
    for i,value in enumerate(prediction):
        if value > 0.5:
            prediction[i] = 1
        else:
            prediction[i] = 0
    tags = tag_encoder.inverse_transform(np.array([prediction]))
    return tags

In [86]:

predict_tags('If NoClassDefFoundError cause ClassNotFoundException Java expect catch throwables When I run code app exit ClassNotFoundException uncaught ClassNotFoundException try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch NoClassDefFoundError e When I attempt compile code compiler complain ClassNotFoundException reachable throw within try-clause try-catch statement Won compile try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch ClassNotFoundException e When I run code throwable catch NoClassDefFoundError catch throwable type javalangNoClassDefFoundError javalangClassNotFoundException cause try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch Throwable e Systemoutprintln egetClass getName Systemoutprintln egetCause getClass getName The follow code compile catch error error clumsy possible workaround try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz 1 0 throw new ClassNotFoundException want code compile catch ClassNotFoundException e Systemoutprintln " ex " catch NoClassDefFoundError e Systemoutprintln " err " And yet I write follow I get away without catch clause cause error yet work fine try throw new Error new IOException catch Error e Systemoutprintln " err " Example 3 would lead conclude throwable NoClassDefFoundError Example 1 would lead conclude throwable ClassNotFoundException And yet Example 2 show java win even let write code properly catch ClassNotFoundException Just I conclude problem error-caused-by-an-exception I run code show previous example show rule Can someone please explain go PS stack trace javalangNoClassDefFoundError commypckageMyClass javalangClassLoaderdefineClass1 Native Method javalangClassLoaderdefineClass ClassLoaderjava 791 Main MyClassLoadergetClasses Mainjava 78 Mainmain Mainjava 109 Caused javalangClassNotFoundException commypckageMyClass javalangClassLoaderfindClass ClassLoaderjava 522 javalangClassLoaderloadClass ClassLoaderjava 423 javalangClassLoaderloadClass ClassLoaderjava 356 4')

[('java',)]

In [87]:

X_test=X_test.reset_index()

In [88]:
X_test.iloc[10,1]

'If NoClassDefFoundError cause ClassNotFoundException Java expect catch throwables When I run code app exit ClassNotFoundException uncaught ClassNotFoundException try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch NoClassDefFoundError e When I attempt compile code compiler complain ClassNotFoundException reachable throw within try-clause try-catch statement Won compile try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch ClassNotFoundException e When I run code throwable catch NoClassDefFoundError catch throwable type javalangNoClassDefFoundError javalangClassNotFoundException cause try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch Throwable e Systemoutprintln egetClass getName Systemoutprintln egetCause getClass getName The follow code compile catch error error clumsy possible workaround try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetNam

In [89]:
Question.loc[Question['Title_Body']=='If NoClassDefFoundError cause ClassNotFoundException Java expect catch throwables When I run code app exit ClassNotFoundException uncaught ClassNotFoundException try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch NoClassDefFoundError e When I attempt compile code compiler complain ClassNotFoundException reachable throw within try-clause try-catch statement Won compile try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch ClassNotFoundException e When I run code throwable catch NoClassDefFoundError catch throwable type javalangNoClassDefFoundError javalangClassNotFoundException cause try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz catch Throwable e Systemoutprintln egetClass getName Systemoutprintln egetCause getClass getName The follow code compile catch error error clumsy possible workaround try Class clazz defineClass null bytes 0 byteslength null tableput clazzgetName clazz 1 0 throw new ClassNotFoundException want code compile catch ClassNotFoundException e Systemoutprintln " ex " catch NoClassDefFoundError e Systemoutprintln " err " And yet I write follow I get away without catch clause cause error yet work fine try throw new Error new IOException catch Error e Systemoutprintln " err " Example 3 would lead conclude throwable NoClassDefFoundError Example 1 would lead conclude throwable ClassNotFoundException And yet Example 2 show java win even let write code properly catch ClassNotFoundException Just I conclude problem error-caused-by-an-exception I run code show previous example show rule Can someone please explain go PS stack trace javalangNoClassDefFoundError commypckageMyClass javalangClassLoaderdefineClass1 Native Method javalangClassLoaderdefineClass ClassLoaderjava 791 Main MyClassLoadergetClasses Mainjava 78 Mainmain Mainjava 109 Caused javalangClassNotFoundException commypckageMyClass javalangClassLoaderfindClass ClassLoaderjava 522 javalangClassLoaderloadClass ClassLoaderjava 423 javalangClassLoaderloadClass ClassLoaderjava 356 4']

,Title,Body,Tags,Title_Body
449031,If NoClassDefFoundError cause ClassNotFoundExc...,When I run code app exit ClassNotFoundExceptio...,"[java, exception]",If NoClassDefFoundError cause ClassNotFoundExc...
